<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
    <font color='red'>To begin: Click anywhere in this cell and press <kbd>Run</kbd> on the menu bar. This executes the current cell and then highlights the next cell. There are two types of cells. A <i>text cell</i> and a <i>code cell</i>. When you <kbd>Run</kbd> a text cell (<i>we are in a text cell now</i>), you advance to the next cell without executing any code. When you <kbd>Run</kbd> a code cell (<i>identified by <span style="font-family: courier; color:black; background-color:white;">In[ ]:</span> to the left of the cell</i>) you advance to the next cell after executing all the Python code within that cell. Any visual results produced by the code (text/figures) are reported directly below that cell. Press <kbd>Run</kbd> again. Repeat this process until the end of the notebook. <b>NOTE:</b> All the cells in this notebook can be automatically executed sequentially by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart and Run All</kbd>. Should anything crash then restart the Jupyter Kernal by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart</kbd>, and start again from the top.
        
</div>

<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
<img src="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/cimcb_logo.png?raw=true" width="180px" align="right" style="padding: 20px">

<a id="introduction"></a>

<h1> Metabolomics Data Visualisation Workflow for ANN-SS</h1>

<br>
<br>
<br>
<p  style="text-align: justify">This Jupyter Notebook described a metabolomics data analysis and visualisation workflow for a 2 layer artificial neural network with layer 1 consisting of multiple neurons (n = 2 to 6) with a sigmoidal activation, and layer 2 (output layer) consisting of a single neuron with a sigmoidal activation function (ANN-SS) for a binary classification outcome.</p>

<p style="text-align: justify">This computational workflow is described using a previously published NMR dataset by <a href="https://www.nature.com/articles/bjc2015414">Chan et al. (2016)</a>.The study compared the urine metabolomic profile comparison across patients characterised as Gastric Cancer (GC; n=43), Benign Gastric Disease (BN; n=40), and Healthy Control (HE; n=40) using 149 named metabolites. For the purpose of this computational workflow, we compare only the GC vs HE samples in a binary discriminant analysis. The deconvolved and annotated data from this study is deposited on <a href="https://www.metabolomicsworkbench.org/">Metabolomics Workbench</a> (Study ID: ST001047), and can be accessed directly via its Project DOI: <a href="http://dx.doi.org/DOI:10.21228/M8B10B">10.21228/M8B10B</a>. The Excel file used in this workflow can be accessed via the following link: <a href="https://github.com/kevinmmendez/MetabViz/blob/master/dynamic/data/ST001047.xlsx?raw=true">ST001047.xlsx</a>.</p>

<p style="text-align: justify">This computational workflow requires a dataset to be in, or converted to, a previously described standardised Excel file format <a href="https://doi.org/10.1007/s11306-019-1588-0">(Mendez et al. 2019)</a>. This format uses the Tidy Data Framework <a href="https://www.jstatsoft.org/index.php/jss/article/view/v059i10/v59i10.pdf">(Wickham, 2014)</a>, where each row represents an observation (e.g. sample) and each column represents a variable (e.g. age or metabolite). Each excel file (per study) contains two sheets; a data sheet and a peak sheet. The data sheet contains the metabolite concentration together with the metadata associated for each observation (requiring the inclusion of the columns: Idx, SampleID, and Class). The peak sheet contains the additional metadata that pertains to the metabolites in the data sheet (requiring the inclusion of the columns: Idx, Name, and Label). The standardisation of this format allows for the efficient re-use of this computational workflow.</p>

<br>
The steps included in this data analysis and visualisation workflow are: 
<br>

1. <a href="#1">Import Packages</a><br>
2. <a href="#2">Load Data & Peak Sheet</a><br>
3. <a href="#3">Extract X & Y</a><br>
4. <a href="#4">Hyperparameter Optimisation</a><br>
    4.1. <a href="#4.1">Plot R² & Q²</a><br>
    4.2. <a href="#4.2">Plot Projections: Full & CV</a><br>
5. <a href="#5">Build Model</a><br>
6. <a href="#6">Permutation Test</a><br>
7. <a href="#7">Bootstrap Resampling of the Model</a><br> 
8. <a href="#8">Model Evaluation</a><br> 
9. <a href="#9">Model Visualisation</a><br> 
    9.1. <a href="#9.1">Plot Projections: in-bag & out-of-bag</a><br>
    9.2. <a href="#9.2">Plot Loadings</a><br>
    9.3. <a href="#9.2">Plot Feature Importance</a><br>
10. <a href="#10">Export Results</a><br>

</div>

<div style="background-color:rgb(255, 250, 250); padding:5px; padding-left: 1em; padding-right: 1em;">
    
<a id="1"></a>
<h2 style="text-align: justify">1. Import Packages</h2>

<p style="text-align: justify"><em>Packages</em> provide additional tools that extend beyond the basic functionality of the Python programming. Prior to usage, <em>packages</em> need to be imported into the Jupyter environment. The following <em>packages</em> need to be imported for this computional workflow:<br></p>

<ul>
<li style="text-align: justify"><a href="http://www.numpy.org/"><code>numpy</code></a>: a standard package primarily used for the manipulation of arrays</li>

<li style="text-align: justify"><a href="https://pandas.pydata.org/"><code>pandas</code></a>: a standard package primarily used for the manipulation of data tables</li>

<li style="text-align: justify"><a href="https://github.com/CIMCB/cimcb"><code>cimcb</code></a>: a library of helpful functions and tools provided by the authors</li>
</ul>

<br>

</div>

In [ ]:
import numpy as np
import pandas as pd
import cimcb as cb

print('All packages successfully loaded')

<div style="background-color:rgb(240,248,255); border: 1px solid lightgrey; padding:5px; padding-left: 1em; padding-right: 1em;">
    
<a id="1"></a>
<h2 style="text-align: justify"><i>Optional: Set Random Seed</i></h2>

<p style="text-align: justify">To reproduce the figures in the research article, set the random seed to 38. When a neural network is first compilied, the weights are initialised. By default in <a href="https://keras.io/initializers/">Keras</a>, the glorot normal initializer (a.k.a. Xavier normal initializer) is used where the weights are randomly drawn from a truncated normal distribution. The seed is used to set reproducible initial weights from this distribution.</p>

<ul>
    <li style="text-align: justify"><p style="text-align: justify"><code>seed</code>: seed the generator using an integer value e.g. 38 (default = None ; no seed set)</li>
</ul>
<br>

</div>

In [ ]:
seed = 56
# seed = None

<div style="background-color:rgb(255, 250, 250); padding:5px; padding-left: 1em; padding-right: 1em;">

<a id="2"></a>
<h2 style="text-align: justify">2. Load Data & Peak Sheet</h2>

<p style="text-align: justify">This CIMCB helper function <code>load_dataXL()</code> loads the <em>Data</em> and <em>Peak</em> sheet from an Excel file. In addition, this helper function checks that the data is in the standardised Excel file format described <a href=#introduction>above</a>. After the initial checks, <code>load_dataXL()</code> outputs two individual Pandas DataFrames (i.e. tables) called <code>DataTable</code> and <code>PeakTable</code> from the Excel file <a href="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/dynamic/data/ST001047.xlsx?raw=true">ST001047.xlsx</a>. This helper function requires values for the following parameters:</p>
<ul>
    <li><code>filename</code>: the name of the excel file (.xlsx file)</li>
    <li><code>DataSheet</code>: the name of the data sheet in the file</li>
    <li><code>PeakSheet</code>: the name of the peak sheet in the file</li>
</ul>   
<br>

</div>

In [ ]:
home = 'data/'
file = 'ST001047.xlsx'

DataTable,PeakTable = cb.utils.load_dataXL(filename=home + file, DataSheet='Data', PeakSheet='Peak')

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="3"></a>
<h2 style="text-align: justify">3. Extract X & Y</h2>

<p style="text-align: justify">Prior to performing any statistical or machine learning modelling, it is best practice to assess the quality of the data and remove metabolites that lack reproducible measurements  <a href="https://link.springer.com/article/10.1007/s11306-018-1367-3">(Broadhurst et al. 2018)</a>. In this dataset <a href="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/dynamic/data/ST001047.xlsx?raw=true">ST001047.xlsx</a>, we can find that the QC-RSD and percentage of missing value has been previously calculated (refer to the peak sheet). In this Jupyter Notebook, we remove all metabolites that do not meet the following criteria:</p>

<ul>
<li style="text-align: justify">QC-RSD less than 20% </li>

<li style="text-align: justify">Fewer than 10% of values are missing</li>
</ul>

<br>
<p style="text-align: justify">The following steps are needed to: <b>(a)</b> extract the binary outcome (i.e. GC vs. HE) and <b>(b)</b> extract, transform, and scale the metabolite data matrix, with missing values imputed.</p>

<ul>
    
<li style="text-align: justify">Create a subset of <code>DataTable</code> called <code>DataTable2</code>, only with samples in the Class “GC” or “HE”</li>
    
<li style="text-align: justify">Set <code>Y</code> to a list (or 1D array) of binary outcomes based on the Class column from <code>DataTable2</code> (“GC”=1 and “HE”=0)</li>

<li style="text-align: justify">Create the variable <code>peaklist</code> to hold the names (M1...Mn) of the metabolites to be used</li>

<li style="text-align: justify">Using this <code>peaklist</code>, extract all corresponding columns (i.e. metabolite data) from <code>DataTable2</code>, and place it in matrix <code>X</code></li>

<li style="text-align: justify">Log-transform the values in <code>X</code></li>

<li style="text-align: justify">Using the helper function <code>cb.utils.scale()</code>, scale the log-transformed data to the unit variance (a.k.a. auto scaling).</li>

<li style="text-align: justify">Impute the missing values by using a <em>k</em>-nearest neighbour approach (with three neighbours) using the helper function <code>cb.utils.knnimpute()</code> to give the final matrix, <code>XTknn</code></li>
    
</ul>

<br>
</div>

In [ ]:
# Clean PeakTable
RSD = PeakTable['QC_RSD']
PercMiss = PeakTable['Perc_missing']
PeakTableClean = PeakTable[(RSD < 20) & (PercMiss < 10)]

# Select Subset of Data
DataTable2 = DataTable[(DataTable.Class == "GC") | (DataTable.Class == "HE")]

# Create a Binary Y Vector 
Outcomes = DataTable2['Class']
Y = [1 if outcome == 'GC' else 0 for outcome in Outcomes]
Y = np.array(Y)

# Extract and Scale Metabolite Data 
peaklist = PeakTableClean['Name']
XT = DataTable2[peaklist]
XTlog = np.log(XT)
XTscale = cb.utils.scale(XTlog, method='auto')
XTknn = cb.utils.knnimpute(XTscale, k=3)

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<a id="4"></a>
<h2 style="text-align: justify"> 4. Hyperparameter Optimisation </h2>

<p style="text-align: justify">The CIMCB helper function <code>cb.cross_val.kfold()</code> is used to carry out <em>k</em>-fold cross-validation (<em>k</em>=5) on a set of ANN-SS models with varying number of neurons (1 to 6) and learning rate (0.01 to 0.05) to determine the optimal hyperparamater values. In <em>k</em>-fold cross-validation, the original dataset is randomly split into k sized folds and subsequently trained for <em>k</em> iterations, where the model is trained on 1 – <em>k</em> folds and tested on the <em>k</em> fold <a href='http://ai.stanford.edu/~ronnyk/accEst.pdf'>(Kohavi 1995)</a>. This helper function requires values for the following parameters:</p>
    
<ul>
    <li><code>model</code>: the class of model used by the function, <code>cb.model.NN_SigmoidSigmoid</code></li>
    <li><code>X</code>: the metabolite data matrix, <code>XTknn</code></li>
    <li><code>Y</code>: the binary outcome vector, <code>Y</code></li>
    <li><code>param_dict</code>: a dictionary, <code>param_dict</code>, that describes all key:value pairs to search, with the key name corresponding to the hyperparameter in the model class and the value as the list of possible values</li>
    <li><code>folds</code>: the number of folds in the <em>k</em>-fold cross validation</li>
    <li><code>n_mc</code>: the number of Monte Carlo repetitions of the <em>k</em>-fold CV</li>
</ul>
<br>
</div>

In [ ]:
# Parameter Dictionary
lr = [0.01,0.02,0.03,0.04,0.05]
neurons = [2, 3, 4, 5, 6]

param_dict = dict(learning_rate=lr,
                  n_neurons=neurons,
                  epochs=400,
                  momentum=0.5,
                  decay=0,
                  loss='binary_crossentropy',
                  seed=seed)

# Initialise
cv = cb.cross_val.KFold(model=cb.model.NN_SigmoidSigmoid,                      
                                X=XTknn,                                 
                                Y=Y,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=10)                              

# Run 
cv.run()  

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<a id="4.1"></a>
<h3 style="text-align: justify"> 4.1. Plot R² & Q²</h3>

<p style="text-align: justify">When <code>cv.plot(metric='r2q2', method='ratio')</code> is run, 6 plots of $R^2$ and $Q^2$ statistics are displayed: (a) heatmap of $R^2$, (b) heatmap of $Q^2$, (c) heatmap of | ($R^2 - Q^2$) / $R^2$ |, (d) | ($R^2 - Q^2$) / $R^2$ | vs. $Q^2$, (e) $R^2$ and $Q^2$ against the learning rate, and (f) $R^2$ and $Q^2$ against the number of neurons. Alternatively, if <code>method='standard'</code>, | $R^2 - Q^2$ | is used instead of | ($R^2 - Q^2$) / $R^2$ | . The optimal number of hyperparameters is selected based on the point of inflection in figure b, or if a clear inflection point is not present, where | ($R^2 - Q^2$) / $R^2$ | = 0.2.  Note, the $R^2$ is the mean coefficient of determination for the full dataset, and the $Q^2$ is the mean coefficient of determination for cross-validated prediction dataset over the 100 Monte Carlo repetitions. When <code>cv.plot(metric='auc')</code> is run, the predictability of the model is presented as area under the ROC curve (AUC), $AUC(full)$ & $AUC(cv)$, a non-parametric alternative to $R^2$ & $Q^2$. The following parameters of <code>cv.plot()</code> can be altered:</p>

<ul>
    <li><code>metric</code>: the metric used for the plots (default = 'r2q2'). Alternative metrics include 'auc', 'acc', 'f1score', 'prec', 'sens', and 'spec'
    <li><code>method</code>: the types of plots displayed (default = 'ratio'). Alternative value is 'standard'
    <li><code>ci</code>: the confidence interval in figure b (default = 95)
</ul>

<br>
</div>

In [ ]:
cv.plot(metric='auc', method='ratio', ci=95)
cv.plot(metric='r2q2', method='ratio', ci=95)

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="4.2"></a>
<h3 style="text-align: justify"> 4.2. Plot Projections: Full & CV </h3>
    
<p style="text-align: justify">When <code>cv.plot_projections()</code> is run, a <em>n</em> x <em>n</em> grid of plots are displayed, where <em>n</em> is the number of neurons in the hidden layer to interrogate. These plots include score plots, distribution plots, and receiver operating characteristic (ROC) curves.</p>

<p style="text-align: justify">There are <em>C</em>(<em>n</em>,<em>2</em>) score plots (i.e. a score plot for every combination of 2 neurons e.g. neuron 1 scores vs. neuron 2 scores). Each score plot include the full scores (as circles) and CV scores (as crosses) coloured by group, as well as the 95% confidence interval ellipses for the full scores (as solid lines) and CV scores (as dashed lines). Additionally, the optimal line of separation (dashed grey line) and orthogonal line (solid grey line) are shown.</p>

<p style="text-align: justify">There are <em>n</em> distribution plots (a distribution plot for each neuron scores). The distribution of the full and CV scores for each corresponding group (i.e. 4 discrete distributions overlayed for 2 groups). Each distribution is calculated using kernel density estimation, a standard non-parametric method used for estimation of a probability density function based on a set of data points <a href="https://ned.ipac.caltech.edu/level5/March02/Silverman/paper.pdf">(Silverman 1986)</a>.</p>

<p style="text-align: justify">There are are <em>C</em>(<em>n</em>,<em>2</em>) ROC curves (a ROC curve for every combination of 2 neurons e.g. neuron 1 scores vs. neuron 2 scores). As the ROC curves are for every combination of 2 neurons, the discrimination is calculated based on optimal separation (i.e. the grey line from the corresponding score plot). For each ROC curve plot there is a ROC curve for the full model (green), and ROC curve for the cv model with 95% confidence intervals (yellow). Additionally, the equal distribution line (dashed black line) is shown.</p>

 <ul>
    <li><code>**optional_arguments</code>: optional arguments to specify model hyperparameters if they are changed in this search e.g. learning_rate=0.02 (except number of components). By default, the max value of each hyperparameter is used (unless specificied).</li>
    <li><code>components</code>: Neurons to plot (default = "all" ; plot all components). Alternatively, list the components to plot e.g. [1,3,4]</li>
    <li><code>plot</code>: Data to show (default = 'ci' ; plot only 95% confidence interval ellipses). Alternative values include 'meanci', 'full', 'cv', and 'all'</li>
    <li><code>label</code>: Add labels to groups in scores plot (default = None ; refers to groups as 0/1)
    <li><code>legend</code>: Show legends for plots (default = 'all'). Alternative values are 'scatter', 'dist', 'roc', and 'none'</li>
 </ul>

<br>
</div>

In [ ]:
cv.plot_projections(learning_rate=0.03,
                    components=[1,2,3],  
                    plot="ci",
                    label=DataTable2.Class,
                    legend="all")

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="5"></a>
<h2 style="text-align: justify"> 5. Build Model </h2>

<p style="text-align: justify">A ANN-SS model using <code>cb.model.NN_SigmoidSigmoid</code> is created and initialised using the optimal hyperparameter values determined in step 4. Following this initialisation, the ANN-SS model is trained using the <code>.train(X, Y)</code> method where the X matrix is <code>XTknn</code> and the Y vector is <code>Y</code>. The implementation of ANN-SS in the <code>cb.model.NN_SigmoidSigmoid</code> class uses using <a href="https://keras.io">Keras</a> with a <a href="http://www.deeplearning.net/software/theano/">Theano backend.</a><p>

<br>
</div>

In [ ]:
# Build Model
model = cb.model.NN_SigmoidSigmoid(learning_rate=0.03, 
                                   n_neurons=2,
                                   epochs=400,
                                   momentum=0.5, 
                                   decay=0, 
                                   loss='binary_crossentropy',
                                   seed=seed)

# Train Model
Ypred = model.train(XTknn, Y)
#Ypred_test = model.test(XTknn) # To test model

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="6"></a>
<h2 style="text-align: justify">6. Permutation Test</h2>

<p style="text-align: justify">After a model has been trained, the <code>.permutation_test()</code> method can be used to assess the reliability of the trained model (after selecting the number of latent variables). For the permutation test, the metabolite data matrix is randomised (permuted or 'shuffled'), while the Y (i.e. outcome) is fixed, and subsequently trained and tested on this randomised data <a href='https://link.springer.com/article/10.1007/s11306-011-0330-3'>(Szymańska et al. 2012)</a>. This process is repeated (in this case, n=100) to construct a distribution to fairly access the model. For a dataset with features that have with no meaningful contribution, we would expect a similar $R^2$ and $Q^2$ to a randomised dataset, while for a dataset with features with meaningful contribution, we would expect a $R^2$ and $Q^2$ significantly higher than that of the randomised dataset. When <code>.permutation_test()</code> is run, 2 plots are displayed: (a) $R^2$ and $Q^2$ against "correlation of permuted data against original data", and (b) probability density functions for $R^2$ and $Q^2$, with the $R^2$ and $Q^2$ values found for the model trained on original data presented as ball-and-stick. The following parameter value of <code>.permutation_test()</code> can be altered: 

<ul>
    <li><code>metric</code>: the metric used for the plots (default = 'r2q2'). Alternative metrics include 'auc', 'acc', 'f1score', 'prec', 'sens', and 'spec'. Multiple metrics can be plotted using a list e.g. ['r2q2', 'auc]
    <li><code>nperm</code>: the number of permutations. (default = 100)
    <li><code>legend</code>: to show legend (default = True). Alternative value is False
</ul>

<br>
</div>

In [ ]:
model.permutation_test(metric=['r2q2', 'auc'],
                       nperm=100,
                       legend=True)

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="7"></a>
<h2 style="text-align: justify"> 7. Bootstrap Resampling of the Model</h2>

<p style="text-align: justify">Bootstrap resampling is a resampling method based on random resampling with replacement, commonly used to provide an estimate of sampling distribution of a test statistic <a href="https://epubs.siam.org/doi/book/10.1137/1.9781611970319?mobileUi=0">(Efron, 1982)</a>. In the context of this workflow, the PLS model from step 5 with its fixed hyperparameter values (i.e. number of LVs = 2) is retrained on the resampled with replacement data (in-bag) and evaluated on the unused data (out-of-bag) for 100 resamples. After the model is evaluated for each boostrap, metrics including the predicted values (ypred), LV scores, LV loadings, and feature importance (VIP and coefficients) are stored and used to calculate 95% confidence intervals. To calculate the 95% confidence intervals, various methods can be used including the basic percentile method, corrected percentile method (a.k.a. bias corrected method), and the commonly used bias-corrected and accelerated (BCA) method. In this example, the BCA  method is used with the class <code>cb.boostrap.BCA</code>. Alternatively, use <code>cb.boostrap.Per</code> to use the percentile method, or <code>cb.bootstrap.CPer</code> for the corrected percentile method. To create and run the <code>bootmodel</code> for any method, the following parameter values need to be set:To create and run the <code>bootmodel</code>, the following parameter values need to be set:
   
 <ul>
    <li><code>model</code>: A model with fixed hyperparameter values for boostrap resampling</li>
    <li><code>bootnum</code>: The number of bootstrap resamples (default = 100)</li>

<br>
</div>

In [ ]:
bootmodel = cb.bootstrap.BCA(model, bootnum=100)
bootmodel.run()

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="8"></a>
<h2 style="text-align: justify"> 8. Model Evaluation </h2>

<p style="text-align: justify">After the <code>bootmodel</code> has been run, the <code>.evaluate()</code> method can be used to provide an estimate of the robustness and a measure of the generalised predictability of the model. There are three plots produced when this method is run including a violin plot, probability density function, and a ROC curve. The violin plots shows the distribution of the median predicted score for the in-bag and out-of-bag (i.e. train and test) by group. The distribution plot shows the probability density function of the median predicted score for the in-bag and out-of-bag (i.e. train and test) by group. The ROC curve shows the ROC curve with the median and 95% CI for the in-bag (green) and the median and 95% CI for the out-of-bag (yellow). The method used to calculate the 95% CI for the in-bag (green) is the class selected in the previous cell. In this example, the bias corrected and accelerated method is used as <code>cb.bootstrap.BCA</code> was used in the previous cell to create <code>bootmodel</code>. The following parameter values in <code>.evaluate()</code> can be altered:
    
<ul>
    <li><code>label</code>: Add labels to groups (default = None ; refer to groups as 0/1)
    <li><code>legend</code>: Show legends for plots (default = 'all'). Alternative values are 'roc', 'dist', 'violin', and 'none'</li>

<br>
</div>

In [ ]:
bootmodel.evaluate(label=DataTable2.Class,
                   legend='all') 

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="9"></a>
<h2 style="text-align: justify"> 9. Model Visualisation </h2>

<br>
</div>

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<a id="9.1"></a>
<h3 style="text-align: justify"> 9.1 Plot Projections: in-bag & out-of-bag </h3>
    
<p style="text-align: justify">After the <code>bootmodel</code> has been run, the <code>.plot_projections()</code> method can be used to visualise the latent variable (LV) scores. When this method is run, a <em>n</em> x <em>n</em> grid of plots are displayed, where <em>n</em> is the number of neurons in the hidden layer. These plots include score plots, distribution plots, and receiver operating characteristic (ROC) curves.</p>

<p style="text-align: justify">There are <em>C</em>(<em>n</em>,<em>2</em>) score plots (i.e. a score plot for every combination of 2 neurons e.g. neuron 1 scores vs. neuron 2 scores). Each score plot include the in-bag scores (as circles) and out-of-bag scores (as crosses) coloured by group, as well as the 95% confidence interval ellipses for the in-bag scores (as solid lines) and out-of-bag scores (as dashed lines). Additionally, the optimal line of separation (dashed grey line) and orthogonal line (solid grey line) are shown.</p>

<p style="text-align: justify">There are <em>n</em> distribution plots (a distribution plot for each neuron scores). The distribution of the in-bag and out-of-bag scores for each corresponding group (i.e. 4 discrete distributions overlayed for 2 groups). Each distribution is calculated using kernel density estimation, a standard non-parametric method used for estimation of a probability density function based on a set of data points <a href="https://ned.ipac.caltech.edu/level5/March02/Silverman/paper.pdf">(Silverman 1986)</a>.</p>

<p style="text-align: justify">There are are C(n,2) ROC curves (a ROC curve for every combination of 2 neurons e.g. neuron 1 scores vs. neuron 2 scores). As the ROC curves are for every combination of 2 neurons, the discrimination is calculated based on optimal separation (i.e. the grey line from the corresponding score plot). For each ROC curve plot there is a ROC curve with the LV score for the initial model with the 95% confidence intervals using the in-bag LV scores (green), and a ROC curve for the out-of-bag LV scores with 95% confidence intervals. The method used to calculate the 95% CI for the in-bag (green) is the class used to create the <code>bootmodel</code>. In this example, the bias corrected and accelerated method is used (<code>cb.bootstrap.BCA</code>). Additionally, the equal distribution line (dashed black line) is shown. 

 <ul>
    <li><code>plot</code>: Data to show in plot (default = "ci" ; plot only 95% confidence interval ellipses). Alternative values include 'meanci', 'ib', 'oob', and 'all'</li>
    <li><code>label</code>: Add labels to groups in scores plot (default = None ; refer to groups as 0/1).
    <li><code>legend</code>: Show legends for plots (default = 'all'). Alternative values are 'scatter', 'dist', 'roc', and 'none'</li>

<br>
</div>

In [ ]:
bootmodel.plot_projections(plot='ib',
                           label=DataTable2.Class,
                           legend='all')

In [ ]:
bootmodel.plot_projections(plot='oob',
                           label=DataTable2.Class,
                           legend='all')

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<a id="9.2"></a>
<h3 style="text-align: justify"> 9.2 Plot Loadings </h3>

<p style="text-align: justify">After the <code>bootmodel</code> has been run, the <code>.plot_loadings()</code> method can be used to visualise the neuron loadings (i.e. weights). When this method is run, <em>n</em> plots are displayed, where <em>n</em> is the number of neurons in the hidden layer. The circles in each loadings plot represent the LV loadings for the initial model. The 95% confidence intervals are calculated using bias-correct (BC) bootstrap method in step 6. Any metabolite loadings with a confidence interval crossing the zero line is considered non-significant to the neuron. This method requires values for the following parameters:</p>
    
<ul>
    <li><code>PeakTable</code>: Cleaned PeakTable from step 3</li>
    <li><code>peaklist</code>: Peaks to include in plot (default = None; include all samples).
    <li><code>plot</code>: To plot the original data or median as circles (default 'data'; plot the original data). Alternative value is 'median'.
    <li><code>ylabel</code>: Name of column in PeakTable to use as the ylabel (default = "Label")
    <li><code>sort</code>: Whether to sort plots in absolute descending order (default = True)</li>
</ul>
   
<br>
</div>

In [ ]:
bootmodel.plot_loadings(PeakTable,
                        peaklist,
                        plot='data',
                        ylabel='Label',  # change ylabel to 'Name' 
                        sort=False)      # change sort to True

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="9.3"></a>
<h3 style="text-align: justify"> 9.3 Plot Feature Importance </h3>

<p style="text-align: justify">After the <code>bootmodel</code> has been run, the <code>.plot_featureimportance()</code> method can be used to visualise the feature importance metrics. When this method is run, <em>2</em> plots are displayed; Connection Weight plot and Garson's Algorithm plot. These feature importance metrics are alternatives to the coefficient and variable importance in projection (VIP) in PLS.
    
<p style="text-align: justify"> The values in the Connection Weight plot contain information about the overall contribution of each metabolite <a href="https://depts.washington.edu/oldenlab/wordpress/wp-content/uploads/2013/03/EcologicalModelling_2004.pdf">(Olden et al. 2004)</a>. The values can either a positive or negative number, and therefore, negatively or positively contribute to the model. Any metabolite coefficient value with a confidence interval crossing the zero line is considered non-significant to the model.</p>
    
<p style="text-align: justify"> The values in the Garson's Algorithm plot contain information about the overall contribution of each metabolite <a href="https://dl.acm.org/citation.cfm?id=129452">(Garson 1991)</a>. These values are absolute, with the higher values representing a higher significance to the model. Unlike in a VIP plot, there is no standard cut-off used to determine whether metabolites are considered "important" in the model.</p>
    
<p style="text-align: justify">This method, <code>bootmodel</code> exports the feature importance metrics as a pandas DataFrame (table). This method also requires values for the following parameters:</p>
    
<ul>
    <li><code>PeakTable</code>: Cleaned PeakTable from step 3</li>
    <li><code>peaklist</code>: Peaks to include in plot (default = None; include all samples).
    <li><code>plot</code>: To plot the original data or median as circles (default 'data'; plot the original data). Alternative value is 'median'.
    <li><code>ylabel</code>: Name of column in PeakTable to use as the ylabel (default = "Label")
    <li><code>sort</code>: Whether to sort plots in absolute descending order (default = True)</li>
</ul>
   
<br>
</div>

In [ ]:
feature_importance = bootmodel.plot_featureimportance(PeakTable,
                                                      peaklist,
                                                      plot='data',
                                                      ylabel='Label',  # change ylabel to 'Name' 
                                                      sort=False)      # change sort to True

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<a id="10"></a>
<h2 style="text-align: justify"> 10. Export Results </h2>

<p style="text-align: justify">The feature importance table created in step 8.3 can be exported using the inbuilt <code>.to_excel()</code> function within a pandas DataFrame. This function requires an input with the name of the file to create, and it can include directories by using the ‘ / ’ symbol. In the cell below, the table <code>feature_importance</code> is exported as an excel file called 'ANNSigSig_ST001047.xlsx' in the 'results' folder.<p>

</div>

In [ ]:
export_folder = 'results/'
export_file = 'ANNSigSig_ST001047.xlsx'

feature_importance.to_excel(export_folder + export_file)
print("Done!")